In [149]:
from datasets import load_dataset, Dataset, DatasetDict, load_from_disk
from bigdata_a3_utils import download_all_amazon_reviews, load_compressed_dataset
from pathlib import Path
import polars as pl
import pandas as pd
from itertools import islice
from typing import Generator

In [150]:
# auto_review = load_dataset(
#     "McAuley-Lab/Amazon-Reviews-2023",
#     "raw_review_Automotive",
#     split="full",
#     trust_remote_code=True,
# )

# auto_meta = load_dataset(
#     "McAuley-Lab/Amazon-Reviews-2023",
#     "raw_meta_Automotive",
#     split="full",
#     trust_remote_code=True,
# )

In [151]:
# auto_meta.to_parquet("amazon_review_meta.parquet")
# auto_review.to_parquet("amazon_review_auto.parquet")

In [152]:
df_review: pl.DataFrame = pl.read_parquet("amazon_review_auto.parquet")
df_meta: pl.DataFrame = pl.read_parquet("amazon_meta_auto.parquet")

In [153]:
df: pl.DataFrame = df_review.join(df_meta, on="parent_asin", how="left")

In [154]:
# Drop invalid ratings (missing or not in [1,2,3,4,5])
df = df.filter(pl.col("rating").is_in([1, 2, 3, 4, 5]))

# Drop empty review texts
df = df.filter(pl.col("text").str.strip_chars().str.len_chars() > 0)

# Fill brand from store or details
df = df.with_columns([
    pl.when(pl.col("store").is_not_null() & (pl.col("store") != ""))
      .then(pl.col("store"))
      .when(pl.col("details").is_not_null())
      .then(
          pl.col("details")
          .str.extract(r"Brand[:\s]*([^\s,;]+)", 1)  # Extract brand from "Brand: XYZ"
          .fill_null("Unknown")
      )
      .otherwise(pl.lit("Unknown"))
      .alias("brand")
])

df = df.with_columns([
    pl.col("text").str.count_matches(r"\b\w+\b").alias("review_length"),
    (pl.col("timestamp").cast(pl.Datetime("ms")).dt.year()).alias("year")
])


df = df.unique(subset=["user_id", "text", "asin"])
df

rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,main_category,title_right,average_rating,rating_number,features,description,price,images_right,videos,store,categories,details,bought_together,subtitle,author,brand,review_length,year
f64,str,str,list[struct[4]],str,str,str,i64,i64,bool,str,str,f64,i64,list[str],list[str],str,struct[4],struct[3],str,list[str],str,str,str,str,str,u32,i32
5.0,"""EASY TO LOAD""","""NICE PERKS FOR THE UPLOAD""",[],"""B00IX1I3G6""","""B00IX1I3G6""","""AHVC24CVCY5RW6GDXZ5TDDT4KS4A""",1547660847272,0,true,null,"""Amazon Reload""",4.7,143309,"[""Add funds to your Amazon Gift Card balance, then use your balance to manage how much you spend while shopping."", ""Save up for a purchase by adding funds to your Gift Card balance."", … ""Amazon Reload purchases are not refundable or redeemable for cash, except as required by law.""]",[],"""None""","{[null, null],[""https://m.media-amazon.com/images/I/41rPcyZJ3nL._AC_.jpg"", ""https://m.media-amazon.com/images/I/41sQl2wriGL._AC_.jpg""],[""https://m.media-amazon.com/images/I/41rPcyZJ3nL._AC_SR38,50_.jpg"", ""https://m.media-amazon.com/images/I/41sQl2wriGL._AC_SR38,50_.jpg""],[""MAIN"", ""PT01""]}","{[""Very easy to do!!""],[""https://www.amazon.com/vdp/0a8f8f3b9f5b409896b362a927a00de1?ref=dp_vse_rvc_0""],[""AGVDC7ODZHCRVYN5D3MC42IBPVJA""]}","""Amazon""","[""Gift Cards"", ""Gift Card Recipients"", ""For Him""]","""{""Item model number"": ""Variabl…",null,null,null,"""Amazon""",5,2019
5.0,"""I Added money to the Amazon g…","""Added$$""",[],"""B00IX1I3G6""","""B00IX1I3G6""","""AG2I6ZK7DE3AA42HOILXPBHQMEDQ""",1548630626249,0,true,null,"""Amazon Reload""",4.7,143309,"[""Add funds to your Amazon Gift Card balance, then use your balance to manage how much you spend while shopping."", ""Save up for a purchase by adding funds to your Gift Card balance."", … ""Amazon Reload purchases are not refundable or redeemable for cash, except as required by law.""]",[],"""None""","{[null, null],[""https://m.media-amazon.com/images/I/41rPcyZJ3nL._AC_.jpg"", ""https://m.media-amazon.com/images/I/41sQl2wriGL._AC_.jpg""],[""https://m.media-amazon.com/images/I/41rPcyZJ3nL._AC_SR38,50_.jpg"", ""https://m.media-amazon.com/images/I/41sQl2wriGL._AC_SR38,50_.jpg""],[""MAIN"", ""PT01""]}","{[""Very easy to do!!""],[""https://www.amazon.com/vdp/0a8f8f3b9f5b409896b362a927a00de1?ref=dp_vse_rvc_0""],[""AGVDC7ODZHCRVYN5D3MC42IBPVJA""]}","""Amazon""","[""Gift Cards"", ""Gift Card Recipients"", ""For Him""]","""{""Item model number"": ""Variabl…",null,null,null,"""Amazon""",1,2019
5.0,"""Great""","""christmas gift""",[],"""B01K8RJDEI""","""B00ADR2LV6""","""AFJNPZQK3NYIJ232JJ52U6NXWS7Q""",1613665005711,0,true,"""Gift Cards""","""Amazon.com Gift Card in a Holi…",4.9,185606,"[""Gift Card is affixed inside a box"", ""Gift amount may not be printed on Gift Cards"", … ""Customized gift message, if chosen at check-out, only appears on packing slip and not on the actual gift card or carrier""]",[],"""25.0""","{[""https://m.media-amazon.com/images/I/81Axm9gA9bL._SL1500_.jpg"", ""https://m.media-amazon.com/images/I/81bjGbl0U3L._SL1500_.jpg"", … null],[""https://m.media-amazon.com/images/I/41S-n8ICtKL.jpg"", ""https://m.media-amazon.com/images/I/41KGkEragiL.jpg"", … ""https://m.media-amazon.com/images/I/31oYO5JWaEL.jpg""],[""https://m.media-amazon.com/images/I/41S-n8ICtKL._SX38_SY50_CR,0,0,38,50_.jpg"", ""https://m.media-amazon.com/images/I/41KGkEragiL._SX38_SY50_CR,0,0,38,50_.jpg"", … ""https://m.media-amazon.com/images/I/31oYO5JWaEL._SX38_SY50_CR,0,0,38,50_.jpg""],[""MAIN"", ""PT01"", … ""PT12""]}","{[""Amazon Gift Box Gift Cards Perfect for any occasion "", ""Holiday Gift Card That Looks Good"", … ""Cool Christmas Gift Card Box""],[""https://www.amazon.com/vdp/04743ba4c0af4f0e8b1d07c6399d65be?ref=dp_vse_rvc_0"", ""https://www.amazon.com/vdp/0547a6e81075493aa539f174f7f90b99?ref=dp_vse_rvc_1"", … ""https://www.amazon.com/vdp/084b1b5864a9416692da36ebe0c45270?ref=dp_vse

In [155]:
# auto_review: DatasetDict = load_dataset(
#     "McAuley-Lab/Amazon-Reviews-2023",
#     "raw_review_Automotive",
#     trust_remote_code=True,
# )

# auto_meta: DatasetDict = load_dataset(
#     "McAuley-Lab/Amazon-Reviews-2023",
#     "raw_meta_Automotive",
#     trust_remote_code=True,
#     streaming=True,
# )

In [156]:
# review_dataset: Dataset = auto_review["full"]
# meta_dataset: Dataset = auto_meta["full"]

In [157]:
# # def stream_to_pandas(dataset: Generator, batch_size=100_000, total=1_000_000) -> pd.DataFrame:
# #     batches: list[pd.DataFrame] = []
# #     it: Generator = iter(dataset)
    
# #     for _ in range(0, total, batch_size):
# #         chunk = list(islice(it, batch_size))
# #         if not chunk:
# #             break
# #         batches.append(pd.DataFrame(chunk))
    
# #     return pd.concat(batches, ignore_index=True)


# def stream_to_pandas(dataset: Generator, batch_size=100_000) -> pd.DataFrame:
#     batches: list[pd.DataFrame] = []
#     it: Generator = iter(dataset)
    
#     while True:
#         # Extract a batch of data
#         chunk = list(islice(it, batch_size))
#         if not chunk:  # Break the loop when the iterator is exhausted
#             break
#         batches.append(pd.DataFrame(chunk))
    
#     # Concatenate all batches into a single DataFrame
#     return pd.concat(batches, ignore_index=True)

In [158]:
# df_review: pd.DataFrame = stream_to_pandas(review_dataset, batch_size=100_000)
# df_meta: pd.DataFrame = stream_to_pandas(meta_dataset, batch_size=100_0000)
# # df = pl.DataFrame(list(islice(meta_dataset, 10)))

In [159]:
# def extract_brand(row) -> str:
#     if row["store"]:
#         return row["store"]  
#     elif row["details"]:
#         for word in row["details"].split():
#             if "Brand" in word:
#                 return word
#     return "Unknown"

In [160]:
# merged_df: pd.DataFrame = pd.merge(df_review, df_meta, on="parent_asin", how="left")
# merged_df = merged_df[
#     merged_df["rating"].notnull() & merged_df["rating"].isin([1, 2, 3, 4, 5])
# ]
# merged_df = merged_df[merged_df["text"].str.strip().str.len() != 0]
# merged_df.loc[:, "brand"] = merged_df.apply(extract_brand, axis=1)
# merged_df['review_length'] = merged_df['text'].apply(lambda x: len(x.split()))
# merged_df['year'] = pd.to_datetime(merged_df['timestamp'], unit='ms').dt.year
# merged_df = merged_df.drop_duplicates(subset=["user_id", "text", "asin"], keep="first")

# df = pl.from_pandas(merged_df)
# df

In [161]:
# df.shape